In [5]:
!pip install pulp
import pulp

     |████████████████████████████████| 40.6MB 106kB/s 


**Zadanie 1**
Minimalizacja strat cieplnych mostka

W zadaniu należy dobrać odpowiednie wartości rezystancji w celu zmianimalizowania strat cieplnych. 

Podane zostały ograniczenia dotyczące maksymalnych prądów w gałęziach mostka:

| Rezystor     | Napięcie[v] |prąd[mA]|
|------------|-------------------||
| R1    | 6              |4|
|R2|10|2|
| R3     | 4              | 2|
| R4       | 7                |2|
| R5 | 3                 |4|

Dla każdego rezystora odchyłka od podanej wartości może wynosić dI = 1mA


In [6]:
# Problem minimalizacji strat cieplnych mostka

from pulp import *
import numpy as np

currents = ['i1','i2','i3','i4','i5']

#[V]
voltages = {'i1': 6.0, 
            'i2':10.0,
            'i3': 4.0,
            'i4': 7.0,
            'i5': 3.0 }

#[mA]
avCurrents = {'i1' : 4.0,
              'i2' : 2.0,
              'i3' : 2.0,
              'i4' : 2.0,
              'i5' : 4.0}

#[mA]
deltas = { 'i1': 1.0,
           'i2': 1.0,
           'i3': 1.0,
           'i4': 1.0,
           'i5': 1.0}

prob = LpProblem("The Bridge Problem", LpMinimize)

i_vars = LpVariable.dicts("currents",currents,0)

prob += lpSum([voltages[i]*i_vars[i] for i in currents])

prob += avCurrents['i1'] + deltas['i1']>= i_vars['i1'] >= avCurrents['i1'] - deltas['i1']
#prob += i_vars['i1'] <= avCurrents['i1'] + deltas['i1']

prob += i_vars['i2'] >= avCurrents['i2'] - deltas['i2']
prob += i_vars['i2'] <= avCurrents['i2'] + deltas['i2']

prob += i_vars['i3'] >= avCurrents['i3'] - deltas['i3']
prob += i_vars['i3'] <= avCurrents['i3'] + deltas['i3']

prob += i_vars['i4'] >= avCurrents['i4'] - deltas['i4']
prob += i_vars['i4'] <= avCurrents['i4'] + deltas['i4']

prob += i_vars['i5'] >= avCurrents['i5'] - deltas['i5']
prob += i_vars['i5'] <= avCurrents['i5'] + deltas['i5']

prob.writeLP("BridgeModel.lp")

prob.solve()
print(str(prob.objective))
results = [];
temp_voltages = []

for v in prob.variables():
    results.append(v.varValue)

for key, value in voltages.items():
     temp_voltages.append(value)    
    
U = np.array(temp_voltages)
    
I = np.array(results)

R = U/I

resistors = ['R1','R2','R3','R4','R5']
res_values = R.tolist()

print("Minimalna wartosc strat mocy:")

print(str(prob.objective.value()) + 'mW\n')

print('Wartosci rezystancji, przy ktorych minimalizowane sa straty mocy:')

for i in range(len(resistors)):
    print(resistors[i]+ ' = ' + str(res_values[i]) + ' kOhm')


6.0*currents_i1 + 10.0*currents_i2 + 4.0*currents_i3 + 7.0*currents_i4 + 3.0*currents_i5
Minimalna wartosc strat mocy:
48.0mW

Wartosci rezystancji, przy ktorych minimalizowane sa straty mocy:
R1 = 2.0 kOhm
R2 = 10.0 kOhm
R3 = 4.0 kOhm
R4 = 7.0 kOhm
R5 = 1.0 kOhm


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


**Zadanie 2**

Maksymalizacja prądu dwójnika. 
W tym zadaniu należy wyznaczyć jaki maksymalny prąd może popłynąć w układzie
mając podane wartości rezystancji oraz maksymalne prądu, które mogą popłynąć 
w danej gałęzi.

**Dane :** 

|     | R |I |     
|------------|-------------------|----------
| R1    |   8      | 2 |
|   R2       |    6               |   3  
|   R3      |    4               |   4  
|   R4      |    10|2
|R5|8|2



In [ ]:
# Problem maksymalizacji pradu dwojnika

from pulp import *
import numpy as np

currents = ['i1','i2','i3','i4','i5']

#[Ohm]
resistances = {'i1': 8.0, 
               'i2': 6.0,
               'i3': 4.0,
               'i4':10.0,
               'i5': 8.0}

#[A]
currMaxes = {'i1': 2.0, 
             'i2': 3.0,
             'i3': 4.0,
             'i4': 2.0,
             'i5': 2.0}

prob = LpProblem("The Current Problem", LpMaximize)

i_vars = LpVariable.dicts("currents",currents,0)

prob += i_vars['i1'] + i_vars['i2']

prob += i_vars['i1'] <= currMaxes['i1']
prob += i_vars['i2'] <= currMaxes['i2']
prob += i_vars['i3'] <= currMaxes['i3']
prob += i_vars['i4'] <= currMaxes['i4']
prob += i_vars['i5'] <= currMaxes['i5']

prob += i_vars['i1'] + i_vars['i2'] == i_vars['i3']
prob += i_vars['i4'] + i_vars['i5'] == i_vars['i3']
prob += i_vars['i1'] + i_vars['i2'] == i_vars['i4'] + i_vars['i5']

prob += i_vars['i1'] * resistances['i1'] == i_vars['i2'] * resistances['i2']
prob += i_vars['i4'] * resistances['i4'] == i_vars['i5'] * resistances['i5']

prob.writeLP("CurrentModel.lp")

prob.solve()

results = [];

for v in prob.variables():
    results.append(v.varValue)

print('Maksymalny prad dwojnika:')

print(str(prob.objective.value()) + 'A\n')

print('Wartosci pradow galezi, przy ktorych maksymalizowany jest prad dwojnika:')

for i in range(len(results)):
    print(currents[i] + ' = ' + str(results[i])  + ' A')


Maksymalny prad dwojnika:
3.6A

Wartosci pradow galezi, przy ktorych maksymalizowany jest prad dwojnika:
i1 = 1.5428571 A
i2 = 2.0571429 A
i3 = 3.6 A
i4 = 1.6 A
i5 = 2.0 A


In [ ]:
!pip install pulp
import pulp

     |████████████████████████████████| 40.6MB 103kB/s 
